# Boargames Data Analysis
In this notebook, we are going to use the dataframe obtained from the Boardgames Scraping class. After some exploratora analysis, we will clean and tranform the data by taking into account the:
- Missing values
- Data types within the different columns
- Categorical data
- Duplicated data

In addition, in order to get a better undestand of the data we will take a look and create data visualization of some characteristics of the dataset.

### Table of Contents

* [Part I](#Exploratory-Data-Analysis) Exploratory Data Analysis
* [Part II](#Clean-Data) Data Cleaning
    * [a)](#Null-Values) Missing values
    * [b)](#Data-types) Data types
    * [c)](#Duplicated-Values) Dupicated values
    * [d)](#Categorical-Variables) Categorical variables
* [Part III](#Data-Visualization) Data Visualization
    * [a)](#Data-Correlation) Features correlation

    

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math 

%matplotlib inline

# import data
df = pd.read_csv('Boardgame/data/boardgame_data.csv')
del df['Unnamed: 0']

# Show df to get an idea of the data
df.head()

# create the new datafram storing the cleaned dataframe
df_cleaned = df.copy()

## <a class="anchor" id="Exploratory-Data-Analysis">Part I : Exploratory Data Analysis</a>

Use the dictionary and cells below to provide some insight into the descriptive statistics of the data.

The boardgames dataset contains the following columns:
 - [0]   Board Game Rank: rank of the game acting as ID
 - [1]   Title
 - [2]   Year: year when the game was released (it can contain string showing the game edition)
 - [3]   Description
 - [4]   Geek Rating: "Bayesian averaging"
 - [5]   Avg Rating: average of all the ratings from registered BGG users
 - [6]   Num Voters: total number of users given rating to the game
 - [7]   Num Players Min
 - [8]   Num Players Max
 - [9]   Best Num Players Min
 - [10]  Best Num Players Max
 - [11]  Playtime Min: minimum allowed playing time
 - [12]  Playtime Max: maximum allowed playing time
 - [13]  Player Min Age
 - [14]  Language Dependence: string showing the dependence of the game to
 - [15]  Weight: Average complexity of this game
 - [16]  Category
 - [17]  Designer

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4900 entries, 0 to 4899
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Board Game Rank       4900 non-null   int64  
 1   Title                 4900 non-null   object 
 2   Year                  4891 non-null   object 
 3   Description           3360 non-null   object 
 4   Geek Rating           4900 non-null   float64
 5   Avg Rating            4900 non-null   float64
 6   Num Voters            4900 non-null   int64  
 7   Num Players Min       4899 non-null   float64
 8   Num Players Max       4899 non-null   float64
 9   Best Num Players Min  4580 non-null   float64
 10  Best Num Players Max  4557 non-null   float64
 11  Playtime Min          4899 non-null   float64
 12  Playtime Max          4899 non-null   float64
 13  Player Min Age        4899 non-null   float64
 14  Language Dependence   4900 non-null   object 
 15  Weight               

In [21]:
df.describe()

,Board Game Rank,Geek Rating,Avg Rating,Num Voters,Num Players Min,Num Players Max,Best Num Players Min,Best Num Players Max,Playtime Min,Playtime Max,Player Min Age,Weight
count,4900.00000,4900.000000,4900.000000,4900.000000,4899.000000,4899.000000,4580.000000,4557.000000,4899.000000,4899.000000,4899.000000,4900.000000
mean,2450.50000,6.171708,7.089906,3046.353265,1.935497,5.271892,3.229039,3.468071,70.323535,112.685650,10.758114,2.361396
std,1414.65249,0.483325,0.553915,6747.121678,0.697311,7.879397,1.289562,1.535260,159.989456,465.923445,2.902751,0.795796
min,1.00000,5.696000,5.810000,136.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,1225.75000,5.802000,6.670000,524.000000,2.000000,4.000000,2.000000,2.000000,30.000000,30.000000,9.000000,1.785714
50%,2450.50000,6.000000,7.040000,1044.000000,2.000000,4.000000,3.000000,4.000000,45.000000,60.000000,12.000000,2.292893
75%,3675.25000,6.393000,7.460000,2496.000000,2.000000,6.000000,4.000000,4.000000,60.000000,100.000000,13.000000,2.893068
max,4900.00000,8.552000,8.990000,103666.000000,8.000000,100.000000,12.000000,31.000000,5400.000000,22500.000000,18.000000,4.844660


### <a class="anchor" id="Observations">Observations</a>
From the statistical table above we can draw some conclusions:
* There are some values which are do not make sence such as having a game with the minimum or maximum number of players equal to zero.
* There are some rows containing null values
* The Year column is not considered as numerical column
* By comparing the 75% and max rows, you can observe big changes in some columns. Are those outliers?
* There are some columns they can be defined and treated as categorical (maybe even creating dummy columns to not impose a rank of categories)

## <a class="anchor" id="Clean-Data">Part II : Clean data</a>
- put the language dependence col as a (needed, semi needed, not needed) -> put it as dummy
- get the type of planes (mysterious, cards...) -> put it as dummy
- check for nan's

### <a class="anchor" id="Invalid-Values">Invalid values</a>
From the describe

In [22]:
# from the table above, we will find those games which has invalid values
# for instance having the number of players or the playtime equal to 0
zero_players_min = df[df['Num Players Min'] == 0].index
zero_players_max = df[df['Num Players Max'] == 0].index
zero_playtime_min = df[df['Playtime Min'] == 0].index
zero_playtime_max = df[df['Playtime Max'] == 0].index
zero_weight = df[df['Weight'] == 0].index # the game complexity goes from 1-5 as defined at the BoardGamesGeek website

In [23]:
# unit testing
all_index = np.concatenate((zero_players_min, zero_players_max))

unique_index = set(all_index)
repetitive_index = set.intersection(set(zero_players_min), set(zero_players_max))

assert len(zero_players_min)+len(zero_players_max) == len(all_index),"Concatenate arrays not correctly built."
assert len(repetitive_index)+len(unique_index) == len(all_index),"Repetitive and Unique arrays not correctly built."

In [24]:
# find which games have invalid values
all_index = np.concatenate((zero_players_min, zero_players_max, zero_playtime_min, zero_playtime_max, zero_weight))
unique_index = set(all_index)
print('There are {} games whose some of their features are invalid'.format(len(unique_index)))

There are 13 games whose some of their features are invalid


In [25]:
# drop indexes with invalid values
df.drop(index=unique_index, axis=0, inplace=True)

In [26]:
assert df[df['Best Num Players Min'] > df['Best Num Players Max']].shape[0] == 0, "Something is bad.."

In [27]:
assert df[df['Num Players Min'] > df['Num Players Max']].shape[0] == 0, "Something is bad.."

In [28]:
assert df[df['Num Players Max'] < df['Best Num Players Max']].shape[0] == 0, "Something is strange.."

In [29]:
#assert df[df['Num Players Min'] > df['Best Num Players Min']].shape[0] == 0, "Something is strange.."

AssertionError: Something is strange..

In [30]:
# games in which the best number of minimal players are not considered as allowed for the game designer
# but why not, I let them inside
df[df['Num Players Min'] > df['Best Num Players Min']]

,Board Game Rank,Title,Year,Description,Geek Rating,Avg Rating,Num Voters,Num Players Min,Num Players Max,Best Num Players Min,Best Num Players Max,Playtime Min,Playtime Max,Player Min Age,Language Dependence,Weight,Category,Designer
414,415,Fantasy Realms,2017,Build the greatest fantasy kingdom by construc...,6.973,7.48,4859,3.0,6.0,2.0,3.0,20.0,20.0,14.0,Moderate in-game text - needs crib sheet or pa...,1.741935,Family,Bruce Glassco
1769,1770,Back to the Future: Back in Time,2020,Reset the 1955 timeline before you disappear a...,6.187,7.43,961,2.0,4.0,1.0,2.0,50.0,50.0,10.0,Some necessary text - easily memorized or smal...,2.259259,Thematic,Prospero Hall
2283,2284,Escape the Room: Mystery at the Stargazer's Manor,2016,NaN,6.036,6.58,1610,3.0,8.0,2.0,3.0,90.0,90.0,10.0,Extensive use of text - massive conversion nee...,1.611111,Family,Rebecca Bleau
2570,2571,Die Legenden von Andor: Chada & Thorn,2015,NaN,5.973,6.72,1141,2.0,2.0,1.0,1.0,45.0,45.0,10.0,Moderate in-game text - needs crib sheet or pa...,2.184211,Thematic,Gerhard Hecht
2613,2614,Escape the Room: Secret of Dr. Gravely's Retreat,2016,NaN,5.965,6.99,737,3.0,8.0,2.0,3.0,90.0,90.0,13.0,Extensive use of text - massive conversion nee...,1.769231,NaN,Rebecca Bleau


### <a class="anchor" id="Null-Values">Null values</a>
Check the missing values in the dataset. 
There are some columns in which a missing value can lead to removing the row whereas others the missing information is not highly relevant.

For instance, it will not be accepted a row with missing _Title_ or _Boardgame Rank_ since those are indexes representing the ID of the boardgame. Noneofthelees, if the Description information is missing we can still use the row.

In [31]:
# The number of missing values in each column of the dataset
df.isnull().mean()*df.shape[0]

Board Game Rank            0.0
Title                      0.0
Year                       6.0
Description             1531.0
Geek Rating                0.0
Avg Rating                 0.0
Num Voters                 0.0
Num Players Min            1.0
Num Players Max            1.0
Best Num Players Min     310.0
Best Num Players Max     331.0
Playtime Min               1.0
Playtime Max               1.0
Player Min Age             1.0
Language Dependence        0.0
Weight                     0.0
Category                 735.0
Designer                  28.0
dtype: float64

In [32]:
# Drop the rows with missing number of players
df.dropna(subset=['Num Players Min','Num Players Max'], axis=0, inplace=True)

In [ ]:
# set of columns with missing values
with_nulls = set(df.columns[df.isnull().mean()!=0])
with_nulls

### <a class="anchor" id="Data-types">Data types</a>
* Title, Description, Designer -> transform to str
* Year -> transform to int
* Language dependece and category -> clean to be categorical

In [ ]:
# convert string columns to string using astype:
# there are some years being Revised Edition -> set them to Nan
# from 9 NaN it turns to be 111 without real year
df['Year_Num'] = pd.to_numeric(df['Year'], errors='coerce')

In [ ]:
print(df[df.Year.isnull() == True].shape)
print(df[df.Year_Num.isnull() == True].shape)

In [ ]:
# Check for strange published years
df[(df.Year_Num < 1900) | (df.Year_Num > 2021)]

In [ ]:
# set invalid years as nan
invalid_years = df[(df.Year_Num < 0) | (df.Year_Num > 2021)].index
df.at[invalid_years, 'Year_Num'] = math.nan
print('Setting invalid years {} as nan'.format(invalid_years))

assert df[(df.Year_Num < 0) | (df.Year_Num > 2021)].shape[0] == 0, "Something went wrong"

### <a class="anchor" id="Duplicated-Values">Duplicated Values</a>

In [ ]:
# check if there are games which are repeated in the dataframe
if(df['Title'].nunique() != df.shape[0]):
    print('Mmmh it looks like some playgames are repeated...')
    duplicated_titles = set([x for x in list(df['Title']) if list(df['Title']).count(x) > 1])

print('There are {} title which are duplicated'.format(len(duplicated_titles)))

In [ ]:
# lets have a look to the first one
df[df['Title'] == list(duplicated_titles)[0]]

### <a class="anchor" id="Categorical-Variables">Categorical variables</a>
One of the main ways for working with categorical variables is using 0, 1 encodings. In this technique, you create a new column for every level of the categorical variable. The advantages of this approach include:

The ability to have differing influences of each level on the response.
You do not impose a rank of the categories.
The ability to interpret the results more easily than other encodings.
The disadvantages of this approach are that you introduce a large number of effects into your model.

In our dataset we can transform to columns as categorical variables:
* _Language Dependence_: representing the degree of dependancy of the language when playing the game
* _Category_: different types of category

In [ ]:
# Identify columns that are categorical in nature
cat_df = df.select_dtypes(include=['object']) # Subset to a dataframe only holding the categorical columns

# Print how many categorical columns
cat_df.columns

In [ ]:
# Add an extra column with language dependence as a categorical variable
df['Language Dependence'].head()

In [ ]:
# divide the language dependence into three categorical values
language_dependece_cat = []
for dependence in df['Language Dependence']:
    if('Unplayable' in dependence):
        language_dependece_cat.append('Extensive')
    elif('Extensive' in dependence):
        language_dependece_cat.append('Extensive')
    elif('Moderate' in dependence):
        language_dependece_cat.append('Moderate')
    elif('Some necessary' in dependence):
        language_dependece_cat.append('Moderate')
    elif('No necessary' in dependence):
        language_dependece_cat.append('No')
    else:
        language_dependece_cat.append(math.nan)

In [ ]:
# insert parsed language dependency column
col_index = df.columns.get_loc('Language Dependence')
df.insert(col_index+1, 'Language', language_dependece_cat, True)

# drop old one
df.drop(labels='Language Dependence', axis=1, inplace=True)

In [ ]:
#Provide a pandas series of the counts for each Language dependence
language_vals = df['Language'].value_counts()

# Bar chart of the language dependency category in percentage
(language_vals/df.shape[0]).plot(kind="bar");
plt.title("Is language a requirement?");

In [ ]:
# For each of the categorical variables create a column
# Ignore NaN columns (dummy_na=False) -> do not create an extra col for the nan values, they are set to 0
print('Language column has {} as unique values'.format(df['Language'].unique()))
language_dummy_df = pd.get_dummies(df['Language'], dummy_na=False).head()
language_dummy_df

In [ ]:
# have a look at the different boardgame categories
df['Category'].head()

In [ ]:
df['Category'].unique()

In [ ]:
# get unique values
cat = []
for categories in df['Category']:
    for category in str(categories).split('|'):
        category = category.strip()
        if (category not in cat):
            cat.append(category)
            
print('There are {} categories: {}'.format(len(cat), cat))

In [ ]:
df['Category'].head()

In [ ]:
category_dummy_df = df['Category'].str.replace(" ","").str.get_dummies("|").head()
category_dummy_df.iloc[10:20, :] 

In [ ]:
cat_all = []
for categories in df['Category']:
    for category in str(categories).split('|'):
        cat_all.append(category.strip())

from collections import Counter
        
#Provide a pandas series of the counts for each Language dependence
cat_vals = Counter(cat_all)

# Bar chart of the language dependency category
plt.title("Boardgames categories");
plt.bar(range(len(cat_vals)), list(cat_vals.values()), align='center')
plt.xticks(range(len(cat_vals)), list(cat_vals.keys()), rotation='vertical')

In [ ]:
# merge categorical columns
df_all_num = pd.concat([df, language_dummy_df, category_dummy_df], axis=1)
df_all_num.head()

In [ ]:
df_all_num.columns

In [ ]:
df_all_num.drop(labels=['Language', 'Category'], axis=1, inplace=True)
df_all_num.columns

In [ ]:
# save cleaned data 
df_all_num.to_pickle("Boardgame//data//boardgame_data_clean.pkl")
print ("Percentage data filtered {:.2f}".format((df.shape[0]-df_all_num.shape[0])/df.shape[0]*100)) 

### <a class="anchor" id="Data-Visualization">Part III : Data Visualization</a>

In [ ]:
def plot_histogram(data):
    ax = plt.gca()
    counts, _, patches = ax.hist(data)
    for count, patch in zip(counts, patches):
        if count>0:
            ax.annotate(str(int(count)), xy=(patch.get_x(), patch.get_height()+5))
    if data.name:
        plt.xlabel(data.name)

In [ ]:
plt.title('Histogram of Average Ratings')
plot_histogram(df['Avg Rating'])

In [ ]:
plt.title('Histogram of Average Weight')
plot_histogram(df['Weight'])

In [ ]:
columns = ['Num Players Min', 'Num Players Max']
plt.figure(figsize=(15, 5))
i=1
for col in columns:
    plt.subplot(1, 2, i)
    plot_histogram(df[col])
    i+=1

In [ ]:
plt.figure(figsize=(15, 5))
fig, axes = plt.subplots(2, 2)
df.boxplot(column='Num Players Min', ax=axes[0,0], figsize=(15, 5)) # add boxplot to 1st subplot
df.boxplot(column='Num Players Max', ax=axes[0,1], figsize=(15, 5)) # add boxplot to 2nd subplot
df.boxplot(column='Best Num Players Min', ax=axes[1,0], figsize=(15, 5)) # add boxplot to 2nd subplot
df.boxplot(column='Best Num Players Max', ax=axes[1,1], figsize=(15, 5)) # add boxplot to 2nd subplot
plt.show()

In [ ]:
# check the number of outliers in this four columns
df_outliers = df[['Num Players Min', 'Num Players Max', 'Best Num Players Min', 'Best Num Players Max']]
Q1 = df_outliers.quantile(0.25)
Q3 = df_outliers.quantile(0.75)
IQR = Q3 - Q1
((df_outliers < (Q1 - 1.5 * IQR)) | (df_outliers > (Q3 + 1.5 * IQR))).sum()

In [ ]:
plt.title('Histogram of Minumum Player Age')
plot_histogram(df['Player Min Age'])

In [ ]:
# for a better understanding of the year, filter those present in the lower quantile
q_low = df['Year_Num'].quantile(0.1)
df_year_filtered = df[df['Year_Num'] > q_low]['Year_Num']

plt.title('Histogram of Year')
plot_histogram(df_year_filtered)

In [ ]:
plt.figure(figsize = (12, 9))
plt.scatter(df['Year_Num'],df['Avg Rating'],alpha=0.6,sizes=(10, 50))
plt.axis([1950, 2021, 5.5, 9.5])
plt.xlabel('Year Published')
plt.ylabel('Average Rating')
plt.show()

### <a class="anchor" id="Data-Correlation">Data Correlation</a>

Let's have a look at how the boardgame features are correlated mongs each other.

In [ ]:
import seaborn as sn

# find correlation between the boardgames features
df_corr = df.drop(labels=['Best Num Players Min', 'Best Num Players Max'], axis=1)
corr_matrix = df_corr.corr()
fig = plt.figure(figsize = (12, 9))
sn.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
# more specificaly display the correlation with the Average Rating column
corr_matrix['Avg Rating'].sort_values(ascending=False)

#### Observations:
- Average Rating is correlated to the weight (complexity of the game) and the minimum age of the player
- Average rating is independent of the playing time, year of publication and number of players

In [ ]:
from matplotlib import pyplot
plt.figure(figsize = (12, 9))
plt.scatter(df['Num Voters'], df['Geek Rating'])
plt.xlabel('Num voters')
plt.ylabel('Average Rating')
plt.show()